#### Download dataset from Kaggle: https://www.kaggle.com/zalando-research/fashionmnist

# 1. Import libraries

In [1]:
import os
import sys
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime

from zipfile import ZipFile
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten

Using TensorFlow backend.


# 2. Extract the downloaded zip package

In [2]:
zip_filepath = "../../../datasets/FashionMNIST/"
zip_filename = "fashionmnist.zip"

In [3]:
if os.path.exists(zip_filepath + zip_filename):
    with ZipFile(zip_filepath + zip_filename, 'r') as zipObj:
        zipObj.extractall(path = "input/")

# 3. Import dataset

In [4]:
train_dataset = pd.read_csv('input/fashion-mnist_train.csv')
test_dataset = pd.read_csv('input/fashion-mnist_test.csv')

## 3.2. Train and Validation

In [5]:
IMG_ROWS, IMG_COLS = 28, 28
CHANNELS = 1
BATCH_SIZE = 1
NUM_CLASSES = 10
TEST_SIZE = 0.2

input_shape = (IMG_ROWS, IMG_COLS, 1)

In [6]:
X = np.array(train_dataset.iloc[:, 1:])
y = np.array(train_dataset.iloc[:, 0])

In [7]:
X.shape, y.shape

((60000, 784), (60000,))

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=TEST_SIZE, random_state=100)

In [9]:
X_train.shape, y_train.shape

((48000, 784), (48000,))

In [10]:
X_val.shape, y_val.shape

((12000, 784), (12000,))

## 3.3. Test

In [11]:
X_test = np.array(test_dataset.iloc[:, 1:])
y_test = np.array(test_dataset.iloc[:, 0])

In [12]:
X_test.shape, y_test.shape

((10000, 784), (10000,))

# 4. Reshape

In [13]:
X_train = X_train.reshape(X_train.shape[0], IMG_ROWS, IMG_COLS, 1)
X_test = X_test.reshape(X_test.shape[0], IMG_ROWS, IMG_COLS, 1)
X_val = X_val.reshape(X_val.shape[0], IMG_ROWS, IMG_COLS, 1)

In [14]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_val = X_val.astype('float32')
X_train /= 255
X_test /= 255
X_val /= 255

In [15]:
class_names = {
                0 : "T-shirt/top", 
                1: "Trouser", 
                2: "Pullover", 
                3: "Dress", 
                4: "Coat",
                5: "Sandal", 
                6: "Shirt", 
                7: "Sneaker", 
                8: "Bag", 
                9: "Ankle Boot"
         }

In [16]:
class_names.values()

dict_values(['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot'])

# 5. Sample Images

In [17]:
random_sample = 0

In [18]:
print("Shape: ", X_train[random_sample].shape)
print("Label: ", y_train[random_sample], "->", class_names[y_train[random_sample]])

Shape:  (28, 28, 1)
Label:  6 -> Shirt


## 5.1. Log image and view in Tensorboard

In [19]:
logdir = "logs/train_data/" + datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.FileWriter(logdir)

In [20]:
img = np.reshape(X_train[random_sample], (-1, IMG_ROWS, IMG_COLS, 1))
img.shape

(1, 28, 28, 1)

In [21]:
with tf.Session() as sess:
    file_writer.add_summary(tf.summary.image("Training data", img).eval(session=sess))

### Run tensorboard --logdir logs/train_data

## 5.2. Log multiple images and view in Tensorboard

In [22]:
images = np.reshape(X_train[0: 25], (-1, IMG_ROWS, IMG_COLS, 1))
images.shape

(25, 28, 28, 1)

In [23]:
with tf.Session() as sess:
    file_writer.add_summary(tf.summary.image("25 Training Samples", images, max_outputs = 25).eval(session=sess))

# 6. Data preprocessing

In [24]:
def preprocess_y_before_train(y_df):
    y_out = tf.keras.utils.to_categorical(y_df, NUM_CLASSES)
    return y_out

In [25]:
y_train_cat = preprocess_y_before_train(y_train)
y_val_cat = preprocess_y_before_train(y_val)
y_test_cat = preprocess_y_before_train(y_test)

# 7. Model Training

## 7.1. 1 Layer Hidden NN with ReLU

### 7.1.1. Train

In [26]:
N_HIDDEN = 1024
BATCH_SIZE = 128
EPOCHS = 50

In [27]:
logdir = "logs/train_data/" + "model1"
tensorboard_callback1 = keras.callbacks.TensorBoard(log_dir=logdir)

In [28]:
model1 = Sequential(name="model1")
model1.add(Dense(N_HIDDEN, input_dim = IMG_ROWS * IMG_ROWS, name="layer1"))
model1.add(Activation('relu', name="layer1_activation"))
model1.add(Dense(NUM_CLASSES, activation='softmax', name="layer2"))

W0812 03:58:08.156594 10392 deprecation_wrapper.py:119] From C:\Users\Ankit\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0812 03:58:08.160590 10392 deprecation_wrapper.py:119] From C:\Users\Ankit\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [29]:
model1.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

W0812 03:58:08.208596 10392 deprecation_wrapper.py:119] From C:\Users\Ankit\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0812 03:58:08.243597 10392 deprecation_wrapper.py:119] From C:\Users\Ankit\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [30]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (None, 1024)              803840    
_________________________________________________________________
layer1_activation (Activatio (None, 1024)              0         
_________________________________________________________________
layer2 (Dense)               (None, 10)                10250     
Total params: 814,090
Trainable params: 814,090
Non-trainable params: 0
_________________________________________________________________


In [31]:
model1_fit = model1.fit(X_train.reshape(-1, IMG_ROWS * IMG_COLS), y_train_cat, 
                        epochs = EPOCHS, batch_size = BATCH_SIZE, validation_data=(X_val.reshape(-1, IMG_ROWS * IMG_COLS), y_val_cat), 
                        callbacks=[tensorboard_callback1])

W0812 03:58:08.395595 10392 deprecation.py:323] From C:\Users\Ankit\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0812 03:58:08.458592 10392 deprecation_wrapper.py:119] From C:\Users\Ankit\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0812 03:58:08.558609 10392 deprecation_wrapper.py:119] From C:\Users\Ankit\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Train on 48000 samples, validate on 12000 samples


W0812 03:58:08.842592 10392 deprecation_wrapper.py:119] From C:\Users\Ankit\Anaconda3\lib\site-packages\keras\callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0812 03:58:08.845594 10392 deprecation_wrapper.py:119] From C:\Users\Ankit\Anaconda3\lib\site-packages\keras\callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/50
48000/48000 [==============================] - 10s 204us/step - loss: 0.5152 - acc: 0.8172 - val_loss: 0.4582 - val_acc: 0.8259
Epoch 2/50
48000/48000 [==============================] - 9s 185us/step - loss: 0.3844 - acc: 0.8619 - val_loss: 0.3586 - val_acc: 0.8733
Epoch 3/50
48000/48000 [==============================] - 9s 177us/step - loss: 0.3394 - acc: 0.8764 - val_loss: 0.3484 - val_acc: 0.8726
Epoch 4/50
48000/48000 [==============================] - 9s 190us/step - loss: 0.3143 - acc: 0.8845 - val_loss: 0.3273 - val_acc: 0.8779
Epoch 5/50
48000/48000 [==============================] - 10s 205us/step - loss: 0.2980 - acc: 0.8900 - val_loss: 0.3394 - val_acc: 0.8761
Epoch 6/50
48000/48000 [==============================] - 10s 198us/step - loss: 0.2775 - acc: 0.8984 - val_loss: 0.3247 - val_acc: 0.8860
Epoch 7/50
48000/48000 [==============================] - 11s 228us/step - loss: 0.2666 - acc: 0.9022 - val_loss: 0.3068 - val_acc: 0.8882
Epoch 8/50
48000/48000 [======

In [32]:
print("Average Loss: ", np.average(model1_fit.history['loss']))

Average Loss:  0.1590765226857364


### 7.1.2. Predict

In [33]:
temp_predict = model1.predict(X_test.reshape(-1, IMG_ROWS * IMG_COLS))

In [34]:
predict1 = []

for index in range(0, len(temp_predict)):
    predict1.append(np.argmax(temp_predict[index]))
    
predict1 = np.asarray(predict1)

In [35]:
print(classification_report(y_test.reshape(-1, 1), predict1.reshape(-1, 1), target_names=class_names.values()))

              precision    recall  f1-score   support

 T-shirt/top       0.82      0.87      0.84      1000
     Trouser       0.99      0.99      0.99      1000
    Pullover       0.84      0.82      0.83      1000
       Dress       0.94      0.89      0.91      1000
        Coat       0.88      0.77      0.82      1000
      Sandal       0.99      0.95      0.97      1000
       Shirt       0.68      0.78      0.73      1000
     Sneaker       0.95      0.95      0.95      1000
         Bag       0.98      0.98      0.98      1000
  Ankle Boot       0.94      0.98      0.96      1000

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000



## 7.2. CNN without Dropout

### 7.2.1. Train

In [36]:
N_HIDDEN = 32
KERNEL_SIZE = 3
BATCH_SIZE = 128
EPOCHS = 50

In [37]:
logdir = "logs/train_data/" + "model2"
tensorboard_callback2 = keras.callbacks.TensorBoard(log_dir=logdir)

In [38]:
model2 = Sequential(name="model2")
model2.add(Conv2D(N_HIDDEN, kernel_size=(KERNEL_SIZE, KERNEL_SIZE), activation="relu", kernel_initializer="he_normal", 
                  input_shape=(IMG_ROWS, IMG_COLS, 1), name="layer1"))
model2.add(MaxPooling2D(pool_size=(2, 2), name="layer2"))
model2.add(Conv2D(N_HIDDEN * 2, kernel_size=(KERNEL_SIZE, KERNEL_SIZE), activation="relu", name="layer3"))
model2.add(MaxPooling2D(pool_size=(2, 2), name="layer4"))
model2.add(Conv2D(N_HIDDEN * 4, kernel_size=(KERNEL_SIZE, KERNEL_SIZE), activation="relu", name="layer5"))
model2.add(Flatten(name="layer6"))
model2.add(Dense(N_HIDDEN * 4, activation="relu", name="layer7"))
model2.add(Dense(NUM_CLASSES, activation='softmax', name="layer8"))

W0812 04:05:43.589227 10392 deprecation_wrapper.py:119] From C:\Users\Ankit\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



In [39]:
model2.compile(optimizer='adam',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

In [40]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
layer2 (MaxPooling2D)        (None, 13, 13, 32)        0         
_________________________________________________________________
layer3 (Conv2D)              (None, 11, 11, 64)        18496     
_________________________________________________________________
layer4 (MaxPooling2D)        (None, 5, 5, 64)          0         
_________________________________________________________________
layer5 (Conv2D)              (None, 3, 3, 128)         73856     
_________________________________________________________________
layer6 (Flatten)             (None, 1152)              0         
_________________________________________________________________
layer7 (Dense)               (None, 128)               147584    
__________

In [41]:
model2_fit = model2.fit(X_train, y_train_cat, 
                        epochs = EPOCHS, batch_size = BATCH_SIZE, validation_data=(X_val, y_val_cat), 
                        callbacks=[tensorboard_callback2])

Train on 48000 samples, validate on 12000 samples
Epoch 1/50
48000/48000 [==============================] - 34s 710us/step - loss: 0.5359 - acc: 0.8034 - val_loss: 0.3832 - val_acc: 0.8611
Epoch 2/50
48000/48000 [==============================] - 36s 749us/step - loss: 0.3436 - acc: 0.8768 - val_loss: 0.3301 - val_acc: 0.8787
Epoch 3/50
48000/48000 [==============================] - 33s 693us/step - loss: 0.2973 - acc: 0.8905 - val_loss: 0.3211 - val_acc: 0.8843
Epoch 4/50
48000/48000 [==============================] - 35s 721us/step - loss: 0.2637 - acc: 0.9026 - val_loss: 0.2855 - val_acc: 0.8984
Epoch 5/50
48000/48000 [==============================] - 32s 659us/step - loss: 0.2395 - acc: 0.9120 - val_loss: 0.2775 - val_acc: 0.9001
Epoch 6/50
48000/48000 [==============================] - 34s 716us/step - loss: 0.2162 - acc: 0.9207 - val_loss: 0.2628 - val_acc: 0.9057
Epoch 7/50
48000/48000 [==============================] - 41s 860us/step - loss: 0.1984 - acc: 0.9265 - val_loss: 0.

In [42]:
print("Average Loss: ", np.average(model2_fit.history['loss']))

Average Loss:  0.08248990760542335


### 7.2.2. Predict

In [43]:
temp_predict = model2.predict(X_test)

In [44]:
predict2 = []

for index in range(0, len(temp_predict)):
    predict2.append(np.argmax(temp_predict[index]))
    
predict2 = np.asarray(predict2)

In [45]:
print(classification_report(y_test.reshape(-1, 1), predict2.reshape(-1, 1), target_names=class_names.values()))

              precision    recall  f1-score   support

 T-shirt/top       0.81      0.90      0.85      1000
     Trouser       0.97      0.99      0.98      1000
    Pullover       0.92      0.79      0.85      1000
       Dress       0.90      0.93      0.92      1000
        Coat       0.87      0.88      0.87      1000
      Sandal       0.99      0.97      0.98      1000
       Shirt       0.76      0.75      0.75      1000
     Sneaker       0.95      0.97      0.96      1000
         Bag       0.99      0.98      0.98      1000
  Ankle Boot       0.96      0.97      0.97      1000

    accuracy                           0.91     10000
   macro avg       0.91      0.91      0.91     10000
weighted avg       0.91      0.91      0.91     10000

